# Use in diffusers

In [16]:
import torch
from diffusers import StableDiffusionXLPipeline
from PIL import Image

from ip_adapter import IPAdapterXL

base_model_path = "/ssdshare/LLMs/stable-diffusion-xl-base-0.9/"
image_encoder_path = "/ssdshare/LLMs/h94-IP-Adapter/sdxl_models/image_encoder/"
ip_ckpt = "/ssdshare/LLMs/h94-IP-Adapter/sdxl_models/ip-adapter_sdxl.bin"
device = "cuda"

# load SDXL pipeline
pipe = StableDiffusionXLPipeline.from_pretrained(
    base_model_path,
    torch_dtype=torch.float16,
    add_watermarker=False,
)


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

In [17]:

# reduce memory consumption
pipe.enable_vae_tiling()

# load ip-adapter
# target_blocks=["block"] for original IP-Adapter
# target_blocks=["up_blocks.0.attentions.1"] for style blocks only
# target_blocks = ["up_blocks.0.attentions.1", "down_blocks.2.attentions.1"] # for style+layout blocks
ip_model = IPAdapterXL(pipe, image_encoder_path, ip_ckpt, "cuda:1", target_blocks=["up_blocks.0.attentions.1"])


  0%|          | 0/30 [00:00<?, ?it/s]

In [31]:
import cv2
import numpy as np
def pil_to_cv2(image_pil):
    image_np = np.array(image_pil)
    image_cv2 = cv2.cvtColor(image_np, cv2.COLOR_RGB2BGR)
    return image_cv2
def resize_img(
    input_image,
    max_side=1280,
    min_side=1024,
    size=None,
    pad_to_max_side=False,
    mode=Image.BILINEAR,
    base_pixel_number=64,
):
    w, h = input_image.size
    if size is not None:
        w_resize_new, h_resize_new = size
    else:
        ratio = min_side / min(h, w)
        w, h = round(ratio * w), round(ratio * h)
        ratio = max_side / max(h, w)
        input_image = input_image.resize([round(ratio * w), round(ratio * h)], mode)
        w_resize_new = (round(ratio * w) // base_pixel_number) * base_pixel_number
        h_resize_new = (round(ratio * h) // base_pixel_number) * base_pixel_number
    input_image = input_image.resize([w_resize_new, h_resize_new], mode)

    if pad_to_max_side:
        res = np.ones([max_side, max_side, 3], dtype=np.uint8) * 255
        offset_x = (max_side - w_resize_new) // 2
        offset_y = (max_side - h_resize_new) // 2
        res[
            offset_y : offset_y + h_resize_new, offset_x : offset_x + w_resize_new
        ] = np.array(input_image)
        input_image = Image.fromarray(res)
    return input_image

In [35]:
prompt = "Create a detailed and refined image of Zero Two from the anime Darling in the Franxx. She is known for her distinctive pink hair and mesmerizing green eyes. She should be depicted in a dynamic pose, showcasing her strong and fearless personality. The image should be in anime style, with an 8k resolution and a 16:9 aspect ratio. The background should be a battlefield, symbolizing the constant fights she has to face. Despite the harsh environment, she maintains a confident and determined expression. The background should be black."
prompt = "Ganyu from Genshin Impact, blue hair and blue eyes, horns and a long ponytail, a blue and white outfit with cloud patterns and gold accents, in an elegant pose, standing in front of a scenic background like mountains, a waterfall, graceful, fantastical, 8k resolution, 16:9 aspect ratio, comic style"

image = "Zero Two.png"
image = Image.open(image)
image.resize((512, 512))

input_image = "sqa.jpg"
input_image = Image.open(input_image)
if input_image is not None:
    input_image = resize_img(input_image, max_side=1024)
    cv_input_image = pil_to_cv2(input_image)
    detected_map = cv2.Canny(cv_input_image, 50, 200)
    canny_map = Image.fromarray(cv2.cvtColor(detected_map, cv2.COLOR_BGR2RGB))
else:
    canny_map = Image.new('RGB', (1024, 1024), color=(255, 255, 255))
    control_scale = 0

# generate image variations with only image prompt
images = ip_model.generate(pil_image=image,
                            prompt="a man, masterpiece, best quality, high quality",
                            negative_prompt= "text, watermark, lowres, low quality, worst quality, deformed, glitch, low contrast, noisy, saturation, blurry",
                            scale=1.0,
                            guidance_scale=5,
                            num_samples=1,
                            num_inference_steps=60, 
                            seed=42,
                            image=canny_map,
                            #neg_content_prompt="a rabbit",
                            #neg_content_scale=0.5,
                          )

images[0].save("result.png")

  0%|          | 0/60 [00:00<?, ?it/s]

# High Resolution Generation

In [ ]:
from hidiffusion import apply_hidiffusion, remove_hidiffusion

# reduce memory consumption
pipe.enable_vae_tiling()

# apply hidiffusion with a single line of code.
apply_hidiffusion(pipe)

...

# generate image at higher resolution
images = ip_model.generate(pil_image=image,
                           prompt="a cat, masterpiece, best quality, high quality",
                           negative_prompt= "text, watermark, lowres, low quality, worst quality, deformed, glitch, low contrast, noisy, saturation, blurry",
                           scale=1.0,
                           guidance_scale=5,
                           num_samples=1,
                           num_inference_steps=30, 
                           seed=42,
                           height=2048,
                           width=2048
                          )
